In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sunidhigupta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Loading and cleaning 

In [2]:
f1 = open("/Users/sunidhigupta/Documents/ML/Week1/hw1q5/d1.txt","r") 
a = f1.read().replace("\n"," ").rstrip()
f2 = open("/Users/sunidhigupta/Documents/ML/Week1/hw1q5/d2.txt","r")
b = f2.read().replace("\n"," ").rstrip()
f3 = open("/Users/sunidhigupta/Documents/ML/Week1/hw1q5/d3.txt","r")
c = f3.read().replace("\n"," ").rstrip()
f4 = open("/Users/sunidhigupta/Documents/ML/Week1/hw1q5/d4.txt","r")
d = f4.read().replace("\n"," ").rstrip()
f5 = open("/Users/sunidhigupta/Documents/ML/Week1/hw1q5/d5.txt","r")
e = f5.read().replace("\n"," ").rstrip()
f_q = open("/Users/sunidhigupta/Documents/ML/Week1/hw1q5/d_query.txt","r")
f = f_q.read().replace("\n"," ").rstrip()

In [3]:
b

"Java is a town of approximately 1,500 people in Georgia in South Ossetia. According to Georgia's current official administrative division, Java is a main town of Java district in the north of Shida Kartli region. According to the South Ossetian side Dzau is an administrative center of Dzau district. The town is situated on the southern slopes of the Greater Caucasus, within the Greater Liakhvi Gorge, 1,040 m (3,412 ft) above sea level.  Java is the second largest urban settlement in South Ossetia, after Tskhinvali. It is located outside the Organization for Security and Co-operation in Europe-defined boundaries of the Georgian-Ossetian conflict zone – an area within a 15-km radius of Tskhinvali.  The town played a major role in the 2008 South Ossetia war, with most of the South Ossetian military forces being located there at the time of the Georgian offensive. During the Battle of Tskhinvali, the government of South Ossetia relocated to Java.  Georgia had accused the Russian military 

In [4]:
bagOfWordsa = a.split(' ')
bagOfWordsb = b.split(' ')
bagOfWordsc = c.split(' ')
bagOfWordsd = d.split(' ')
bagOfWordse = e.split(' ')
bagOfWordsf = f.split(' ')

In [5]:
print(bagOfWordsc)

['USS', 'Java', 'was', 'a', 'wooden-hulled,', 'sailing', 'frigate', 'in', 'the', 'United', 'States', 'Navy,', 'bearing', '44', 'guns.', 'She', 'was', 'named', 'for', 'the', 'American', 'victory', 'over', 'HMS', 'Java', 'off', 'the', 'coast', 'of', 'Brazil', 'on', '29', 'December', '1812,', 'captured', 'by', 'the', 'Constitution', 'under', 'the', 'command', 'of', 'Captain', 'William', 'Bainbridge.', 'HMS', 'Java', 'had', 'suffered', 'severe', 'damage', 'during', 'the', 'engagement', 'and', 'being', 'far', 'from', 'home', 'port', 'was', 'ordered', 'burned.', '', 'Java', 'was', 'built', 'at', 'Baltimore,', 'Maryland', 'in', '1814', 'and', '1815', 'by', 'Flannigan', 'and', 'Parsons.', 'Not', 'completed', 'until', 'after', 'the', 'end', 'of', 'the', 'War', 'of', '1812,', 'Java,', 'Captain', 'Oliver', 'Hazard', 'Perry', 'in', 'command,', 'got', 'underway', 'from', 'Baltimore', '5', 'August', '1815,', 'picked', 'up', 'spare', 'rigging', 'at', 'Hampton', 'Roads', 'and', 'New', 'York,', 'and', 

In [6]:
uniqueWords = set(bagOfWordsa).union(set(bagOfWordsf)).union(set(bagOfWordsb)).union(set(bagOfWordsc)).union(set(bagOfWordsd)).union(set(bagOfWordse))
len(uniqueWords)

643

In [7]:

numOfWordsa = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsa:
    numOfWordsa[word] += 1

numOfWordsb = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsb:
    numOfWordsb[word] += 1
    
numOfWordsc = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsc:
    numOfWordsc[word] += 1
    
numOfWordsd = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsd:
    numOfWordsd[word] += 1
    
numOfWordse = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordse:
    numOfWordse[word] += 1
    
numOfWordsf = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsf:
    numOfWordsf[word] += 1

### Function to compute term frequency

In [8]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [9]:
tfA = computeTF(numOfWordsa, bagOfWordsa)
tfB = computeTF(numOfWordsb, bagOfWordsb)
tfC = computeTF(numOfWordsc, bagOfWordsc)
tfD = computeTF(numOfWordsd, bagOfWordsd)
tfE = computeTF(numOfWordse, bagOfWordse)
tfF = computeTF(numOfWordsf, bagOfWordsf)

### Function to calculate IDF value for each word

In [10]:
def computeIDF(documents):
    import math
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

In [11]:
idfs = computeIDF([numOfWordsa, numOfWordsb, numOfWordsc,numOfWordsd, numOfWordse, numOfWordsf])

In [12]:
def computeTFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [13]:
tfidfA = computeTFIDF(tfA, idfs)
tfidfB = computeTFIDF(tfB, idfs)
tfidfC = computeTFIDF(tfC, idfs)
tfidfD = computeTFIDF(tfD, idfs)
tfidfE = computeTFIDF(tfE, idfs)
tfidfF = computeTFIDF(tfF, idfs)
df = pd.DataFrame([tfidfA, tfidfB, tfidfC,tfidfD,tfidfE,tfidfF])

In [14]:
df

,,captured,syrupy,"city,",ordered,"war,",early,body,"Tskhinvali,","once,",...,offensive.,(which,wet,active,9,face,forces,"sacks,","Pancur),",libraries
0,0.004954,0.000000,0.000000,0.009738,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.002509,0.000000,0.000000,0.000000,0.000000,0.016438,0.000000,0.000000,0.016438,0.000000,...,0.008219,0.000000,0.000000,0.000000,0.000000,0.000000,0.008219,0.000000,0.000000,0.000000
2,0.002272,0.005582,0.000000,0.000000,0.005582,0.000000,0.005582,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.005582,0.000000,0.005582,0.000000,0.000000,0.000000,0.000000
3,0.003364,0.000000,0.006612,0.000000,0.000000,0.000000,0.000000,0.006612,0.000000,0.000000,...,0.000000,0.000000,0.006612,0.000000,0.000000,0.000000,0.000000,0.006612,0.006612,0.000000
4,0.001599,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007859,...,0.000000,0.007859,0.000000,0.000000,0.007859,0.000000,0.000000,0.000000,0.000000,0.007859
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [15]:
type(df.iloc[[5]])
df.iloc[[5]]

,,captured,syrupy,"city,",ordered,"war,",early,body,"Tskhinvali,","once,",...,offensive.,(which,wet,active,9,face,forces,"sacks,","Pancur),",libraries
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
import scipy
new_df = scipy.sparse.csr_matrix(df.values)

res = cosine_similarity(new_df,new_df[5])

In [17]:
res

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.12265387],
       [0.        ],
       [1.        ]])

References:
https://machinelearningmastery.com/how-to-generate-random-numbers-in-python/
https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76